In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os,glob
from os import listdir,makedirs
from os.path import isfile,join
from tqdm import tqdm
import random
from sklearn.model_selection import KFold
import statistics as st


train_dir = '/content/drive/MyDrive/test'
catagories = []
for c in os.listdir(train_dir):
    catagories.append(c)
print(catagories)

['ship', 'horse', 'frog', 'automobile', 'bird', 'dog', 'deer', 'truck', 'airplane', 'cat']


In [3]:
train_data = []
for c in catagories:
    path = os.path.join(train_dir, c)
    label = catagories.index(c)
    for img in tqdm(os.listdir(path)):
        img_path = os.path.join(path, img)
        img_arr = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
        img_arr = cv2.resize(img_arr, (32, 32))
        train_data.append([img_arr, label])

print(f"Total Train Data: {len(train_data)}")

100%|██████████| 1000/1000 [00:17<00:00, 55.59it/s]

Total Train Data: 10000


In [ ]:
random.shuffle(train_data)

KFactor = [0.25, 0.5, 1, 2, 4]
kf = KFold(n_splits=5, random_state=None, shuffle=False)
for i,(train_index, test_index) in enumerate(kf.split(train_data)):
    k_values = np.multiply(np.sqrt(train_index.size),KFactor)
    # k_values = np.arange(1,np.sqrt(train_index.size)*10,40)
    k_values = [int(x) for x in k_values]
    break

In [17]:
accuracy = np.zeros((2,len(k_values),kf.n_splits), dtype=int)

res = []
for i,(train_index, test_index) in enumerate(kf.split(train_data)):
    for x in tqdm(test_index):
        r = []
        testData = train_data[x][0]
        l1 = []
        l2 = []
        for y in train_index:
            trainData = train_data[y][0]
            d = np.sum(abs(testData-trainData))
            l1.append(d)
            d = np.sqrt(np.sum((testData-trainData)**2))
            l2.append(d)
        l1_i = np.argsort(l1)
        l2_i = np.argsort(l2)
        for k in k_values:
            comp = []
            for j in l1_i[:k]:
                comp.append(train_data[j][1])
            if st.mode(comp) == train_data[x][1]:
                accuracy[0][k_values.index(k)][i] += 1
            comp = []
            for j in l2_i[:k]:
                comp.append(train_data[j][1])
            if st.mode(comp) == train_data[x][1]:
                accuracy[1][k_values.index(k)][i] += 1

100%|██████████| 2000/2000 [06:03<00:00,  5.51it/s]
